In [1]:
puzzle = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
len(puzzle)

81

In [2]:
rows = 'ABCDEFGHI'
cols = '123456789'
assert len(rows) == len(cols)

In [3]:
def cross(a, b):
    return [s+t for s in a for t in b]
cross('abc', 'def')

['ad', 'ae', 'af', 'bd', 'be', 'bf', 'cd', 'ce', 'cf']

### All Rows and Columns
Getting all rows and columns using cross(a, b)

In [4]:
boxes = cross(rows, cols)
assert len(boxes)==81
#boxes

Units

In [5]:
row_units = [cross(r, cols) for r in rows]
assert len(row_units) == 9
#row_units

In [6]:
col_units = [cross(rows, c) for c in cols]
assert len(row_units) == 9
#col_units

In [7]:
square_units = [cross(row, col) for row in ['ABC', 'DEF', 'GHI'] for col in ['123', '456', '789']]
assert len(square_units) == 9
#square_units

In [8]:
right_diag = [[rows[i] + cols[i] for i in range(len(rows))]]
left_diag = [[rows[i] + cols[len(rows)-i-1] for i in range(len(rows))]]
assert len(right_diag) == len(left_diag)

In [9]:
all_units = row_units + col_units + square_units + right_diag + left_diag
assert len(all_units) == 29
#unit_list[:10]

### Units
A dict where each key is a box and value is a list of all rows, cols, and 3x3 square that intersect that box.

In [10]:
units = dict((i, [s for s in all_units if i in s]) for i in boxes)
assert len(units) == 81
#units['D1'][0]

### Peers
A dict where each key is a box and value is a list of all rows, cols, nd 3x3 square that interest that box without containing that box.

In [11]:
peers = dict((s, set(sum(units[s], []))-set([s])) for s in boxes)
#peers['D1']

### Collect all values
Modify grid values such that:
key : Grid index (boxes)
value: All possible values for the box

In [12]:
def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args:
        grid(string) - A grid in string form.
    Returns:
        A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    s_grid = dict(zip(boxes, grid))
    digits = '123456789'
    for k,v in s_grid.items():
        if v == '.':
            s_grid[k] = digits
    return s_grid

In [13]:
def display(values):
    """
    Display the values as a 2-D grid.
    Args:
        values(dict): The sudoku in dictionary form
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

### Elimination
This method selects a box and removes it's value from all of it's peers. This method can be rerun N times to solve* the sudoku.

In [14]:
def eliminate(values):
    """
    Eliminate values from current pool with the given values
    Args:
        values(dict) - Current state of the sduoku
                       Key: Box index, V: Current value in that box
    Returns:
        values(dict) - New state
    """
    given_value_location = [k for k, v in values.items() if len(v) == 1]
    for i in given_value_location:
        current_value = values[i]
        for j in peers[i]:
            values[j] = values[j].replace(current_value, '')
    return values

In [15]:
def only_choice(values):
    """
    There may be only one possible choice for a particular unit
    Args:
        values(dict) - Current state of the sduoku
                       Key: Box index, V: Current value in that box
    Returns:
        values(dict) - New state        
    """
    for i in all_units:
        for d in '123456789':
            choices = [j for j in i if d in values[j]]
            if len(choices) == 1:
                values[choices[0]] = d

    return values

In [27]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """
    
    # Find all instances of naked twins
    possible_twins = [k for k,v in values.items() if len(v) == 2]
    twins = [(i, j) for i in possible_twins for j in possible_twins[1:] if values[i] == values[j]]
    # Eliminate the naked twins as possibilities for their peers

    for unit in all_units:
        for twin in twins:
            if twin[0] in unit and twin[1] in unit:
                d1 = values[twin[0]][0]
                d2 = values[twin[0]][1]
                for i in unit:
                    if i != twin[0] or i != twin[1]:
                        values[i] = values[i].replace(d1, '')
                        values[i] = values[i].replace(d2, '')
    return values

In [30]:
#grid = grid_values(puzzle)
#grid = eliminate(grid)
#grid = only_choice(grid)
grid = naked_twins(grid)
display(grid)

IndexError: string index out of range

### Testing
Elimination and only_choice together can solve the sudoku, let's test them out

In [17]:
def reduce_puzzle(values):
    stuck = False
    while not stuck:
        begin = len([i for i in boxes if len(values[i]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        #values = naked_triple(values)
        end = len([i for i in boxes if len(values[i]) == 1])

        sanity = len([i for i in boxes if len(values[i]) == 0])
        if not sanity:
            return

        stuck = begin == end
    return values

In [18]:
def search(values):
    values = reduce_puzzle(values)

    if not values:
        return False

    if all(len(v) == 1 for k, v in values.items()):
        return values

    num, idx = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    pdb.set_trace()
    for i in values[idx]:
        temp_sudoku = values.copy()
        temp_sudoku[idx] = i
        solving = search(temp_sudoku)
        if solving:
            return values

In [19]:
def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    values = grid_values(grid)
    return search(values)

In [23]:
display(solve(puzzle))

> <ipython-input-22-371f3491ae74>(16)naked_twins()
-> for unit in all_units:
(Pdb) display(values)
display (values): {'A1': '2', 'A2': '356789', 'A3': '34789', 'A4': '1345789', 'A5': '134578', 'A6': '134579', 'A7': '13569', 'A8': '1345689', 'A9': '145', 'B1': '45789', 'B2': '57', 'B3': '34789', 'B4': '1345789', 'B5': '134578', 'B6': '6', 'B7': '2', 'B8': '1345', 'B9': '14589', 'C1': '45689', 'C2': '35689', 'C3': '1', 'C4': '34589', 'C5': '23458', 'C6': '23459', 'C7': '35', 'C8': '7', 'C9': '45689', 'D1': '14579', 'D2': '12579', 'D3': '6', 'D4': '457', 'D5': '123457', 'D6': '8', 'D7': '1359', 'D8': '12359', 'D9': '1259', 'E1': '3', 'E2': '1258', 'E3': '248', 'E4': '145', 'E5': '9', 'E6': '1245', 'E7': '156', 'E8': '12568', 'E9': '7', 'F1': '15789', 'F2': '125789', 'F3': '2789', 'F4': '6', 'F5': '12357', 'F6': '57', 'F7': '4', 'F8': '123589', 'F9': '12589', 'G1': '1679', 'G2': '4', 'G3': '2', 'G4': '13579', 'G5': '13567', 'G6': '13579', 'G7': '8', 'G8': '12569', 'G9': '12569', 'H1': '167

BdbQuit: 

In [ ]:
harder_s = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
hard_grid = grid_values(harder_s)
print_grid(solve_sudoku(hard_grid))

### We got problems
It seems that we can't get any further based on our current algorithms so we have to try each possibility.
Let's first start by chosing one of the smallest grid to branch out from

In [ ]:
grid_status = solve_sudoku(hard_grid)
n, idx = min((len(grid_status[i]), i) for i in boxes if len(grid_stats[i]) > 1)
n, idx